**Plot the histograms for general investigation and visual inspection**  
For 2D histograms (as opposed to plot_histograms.ipynb which is for 1D histograms only)  

In [ ]:
### imports

# external modules
import sys
import numpy as np
import matplotlib.pyplot as plt
import importlib

# local modules
sys.path.append('../utils')
import csv_utils as csvu
import dataframe_utils as dfu
import plot_utils as pu
import hist_utils as hu
importlib.reload(csvu)
importlib.reload(dfu)
importlib.reload(pu)
importlib.reload(hu)

In [ ]:
# load a dataframe containing histograms
# note: you might need to change the path or file name, depending on where you have a valid csv file stored!
dffile = '../data/DF2017B_clusterposition_zphi_ontrack_PXLayer_1_run297289.csv'
histdf_all = csvu.read_csv(dffile)
print('number of lumisections: '+str(len(histdf_all)))

# select a single run
#runs = dfu.get_runs(histdf_all)
#print('number of runs: '+str(len(runs)))
#print(runs) # uncomment this to see a printed list of available runs
#runnbs = [297056] # you can also add multiple runs to the list to plot them all together
#histdf = dfu.select_runs(histdf_all,runnbs)

# select DCS-bit on data
histdf = dfu.select_dcson(histdf_all)

print('number of selected lumisections: '+str(len(histdf)))

In [ ]:
# extract the histograms as a numpy array from the dataframe

(histograms,runnbs,lsnbs) = dfu.get_hist_values(histdf)
print('shape of histogram array: {}'.format(histograms.shape))
print('shape of run number array: {}'.format(runnbs.shape))
print('shape of lumisection number array: {}'.format(lsnbs.shape))

# additional info, not needed to get the histograms but may contain useful info
xmin = histdf.at[0,'Xmin']
xmax = histdf.at[0,'Xmax']
nxbins = histdf.at[0,'Xbins']
ymin = histdf.at[0,'Ymin']
ymax = histdf.at[0,'Ymax']
nybins = histdf.at[0,'Ybins']

In [ ]:
# make a plot of a single histogram

index = 10
hist = histograms[index]
pu.plot_hist_2d(hist,title='a histogram title',xaxtitle='x axis title',yaxtitle='y axis title')

In [ ]:
# make a plot of multiple histograms next to each other

_ = pu.plot_hists_2d(histograms[:9],ncols=3)

In [ ]:
# make a gif showing the evolution over time 

figname = 'temp_gif.gif'
titles = []
for lsnb,runnb in zip(lsnbs,runnbs):
    titles.append('run {}, lumisection {}'.format(runnb,lsnb))
pu.plot_hists_2d_gif(histograms,titles=titles,figname=figname)
# note: to view the result, you can directly open the file saved as figname in swan, it will show the animated gif
# (or you can download it and view it with whatever gif viewer you normally use)

In [ ]:
# cropping, rebinning, normalization and averaging

print('original histograms:')
_ = pu.plot_hists_2d(histograms[:4],ncols=4)
plt.show()

# cropping:
# (note: here you can also just manually slice the array, but the dedicated function shown here might be useful in more involved workflows)
print(histograms.shape)
modhists = hu.crophists(histograms,[slice(1,-1,None),slice(1,-1,None)]) # remove under- and overflow bins
print(modhists.shape)
modhists = hu.crophists(modhists,[slice(None,None,None),slice(80,220,None)]) # cut out uninteresting parts
print(modhists.shape)
print('cropped histograms:')
_ = pu.plot_hists_2d(modhists[:4],ncols=4)
plt.show()

# rebinning:
print(modhists.shape)
modhists = hu.rebinhists(modhists,(2,2))
print(modhists.shape)
print('rebinned histograms:')
_ = pu.plot_hists_2d(modhists[:4],ncols=4)
plt.show()

# normalizing:
modhists = hu.normalizehists(modhists)
print('normalized histograms:')
_ = pu.plot_hists_2d(modhists[:4],ncols=4)
plt.show()

# averaging
avghists = hu.averagehists(modhists,1)
print('average histogram(s):')
_ = pu.plot_hists_2d(avghists[:min(4,len(avghists))],ncols=4)
plt.show()

In [ ]:
# equivalent way to obtain the same histograms but with less typing

(histograms) = hu.preparedatafromdf(histdf, 
                                    cropslices=[slice(1,-1,None),slice(81,221,None)], 
                                    rebinningfactor=(2,2), 
                                    donormalize=True, 
                                    doplot=False)
_ = pu.plot_hists_2d(histograms[:4],ncols=4)